In [1]:
import pandas as pd
from pycaret.regression import *

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
train = pd.read_csv("train_meanprice.csv")
test= pd.read_csv("test_meanprice.csv")

In [3]:
train

,Month,shopID,itemID,MeanPrice,n
0,1,0,1000001,420.000000,6
1,1,0,1000002,250.000000,2
2,1,0,1000003,226.941896,0
3,1,0,1000004,120.000000,1
4,1,0,1000005,220.000000,8
...,...,...,...,...,...
67315,10,17,3300001,1775.952381,0
67316,10,17,3400001,850.000000,1
67317,10,17,3400002,1672.455521,0
67318,10,17,3400003,1190.000000,2


In [4]:
exp1 = setup(train, target="n", normalize=True, categorical_features=["Month", "shopID", "itemID"], train_size=0.8, profile=True)

,Description,Value
0,Session id,6375
1,Target,n
2,Target type,Regression
3,Original data shape,"(67320, 5)"
4,Transformed data shape,"(67320, 32)"
5,Transformed train set shape,"(53856, 32)"
6,Transformed test set shape,"(13464, 32)"
7,Numeric features,1
8,Categorical features,3
9,Preprocess,True


Loading profile... Please Wait!
Profiler Failed. No output to show, continue with modeling.


In [5]:
lgbm = create_model("lightgbm")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.5006,1.4006,1.1835,0.5254,0.3130,0.4480
1,0.4914,1.2644,1.1244,0.5357,0.3124,0.4542
2,0.4971,1.9617,1.4006,0.4051,0.3106,0.4573
3,0.4954,1.9644,1.4016,0.4451,0.3096,0.4470
4,0.5004,1.7663,1.3290,0.4087,0.3146,0.4557
5,0.5236,1.5698,1.2529,0.4757,0.3250,0.4646
6,0.4960,1.0846,1.0414,0.5029,0.3212,0.4538
7,0.4851,1.2917,1.1365,0.5337,0.3059,0.4424
8,0.4763,1.0166,1.0083,0.5564,0.3106,0.4481


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
top3 = compare_models(sort='RMSE', n_select=3, fold=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.4665,1.3883,1.1728,0.5173,0.2901,0.4486,19.7600
xgboost,Extreme Gradient Boosting,0.4180,1.4005,1.1795,0.5117,0.2545,0.4569,17.9400
lightgbm,Light Gradient Boosting Machine,0.5014,1.5561,1.2428,0.4583,0.3169,0.4570,1.4267
rf,Random Forest Regressor,0.4077,1.6083,1.2657,0.4375,0.2796,0.4615,10.9700
gbr,Gradient Boosting Regressor,0.6850,2.0202,1.4189,0.2933,0.4289,0.4526,5.4233
dt,Decision Tree Regressor,0.4327,2.2417,1.4965,0.2093,0.3299,0.5158,0.7367
lr,Linear Regression,0.8836,2.7165,1.6469,0.0475,0.5481,0.4856,0.9567
lar,Least Angle Regression,0.8836,2.7169,1.6470,0.0474,0.5482,0.4865,0.5433
br,Bayesian Ridge,0.8839,2.7169,1.6470,0.0474,0.5481,0.4846,0.7533
ridge,Ridge Regression,0.8836,2.7169,1.6470,0.0474,0.5482,0.4865,0.5233


Processing:   0%|          | 0/87 [00:00<?, ?it/s]

In [7]:
stack_model = stack_models(estimator_list=top3, meta_model=lgbm, fold=5, choose_better=True, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3593,1.0254,1.0126,0.6386,0.2309,0.4737
1,0.3684,1.5715,1.2536,0.5404,0.2264,0.4894
2,0.3831,1.4564,1.2068,0.5131,0.2413,0.4828
3,0.3639,0.9732,0.9865,0.6069,0.2338,0.4775
4,0.3558,0.8378,0.9153,0.6693,0.2336,0.4870
Mean,0.3661,1.1729,1.0750,0.5937,0.2332,0.4821
Std,0.0095,0.2875,0.1315,0.0587,0.0049,0.0058


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
final_model = finalize_model(stack_model)

In [9]:
predict = predict_model(final_model, data=test)

In [10]:
predict

,Month,shopID,itemID,MeanPrice,prediction_label
0,10,0,1000001,363.369720,0.003392
1,10,1,1000001,363.369720,0.003392
2,10,2,1000001,363.369720,0.003392
3,10,3,1000001,363.369720,2.775614
4,10,4,1000001,363.369720,1.970386
...,...,...,...,...,...
3055,10,13,3500001,416.063843,0.967141
3056,10,14,3500001,416.063843,0.014349
3057,10,15,3500001,416.063843,0.014349
3058,10,16,3500001,416.063843,0.006374


In [11]:
submit = predict["prediction_label"]

In [12]:
submit

0       0.003392
1       0.003392
2       0.003392
3       2.775614
4       1.970386
          ...   
3055    0.967141
3056    0.014349
3057    0.014349
3058    0.006374
3059    0.028037
Name: prediction_label, Length: 3060, dtype: float64

In [13]:
submit.to_csv("submit4.csv", index=True, header=False)